# visualization of buildouts




In [1]:
import pandas as pd
import numpy as np
import os
import math
#import matplotlib.pyplot as plt
import glob
import os


# show multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", 100)

# plotting (optional)
import cufflinks as cl

cl.go_offline()

from IPython.display import Image



<IPython.core.display.Javascript object>

In [2]:
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd
import shapely.geometry
import numpy as np
import wget

<IPython.core.display.Javascript object>

In [3]:
%load_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

In [4]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [5]:
# import geopandas as gpd
# import contextily as ctx
# import matplotlib.pyplot as plt
# import shapely

# # same as above but using shapely.Point
# from shapely.geometry import Point, LineString, Polygon

# # import numpy as np
# from powerscenarios.parser import Parser
# from powerscenarios.grid import Grid

# from ecmwfapi import ECMWFDataServer
# import netCDF4

# from rex import WindX
# from rex import Resource
# import rex
# import os

# # speeds up tab autocompletion significantly
# %config Completer.use_jedi = False

# from dash import Dash, html, dash_table, dcc, callback, Output, Input

<IPython.core.display.Javascript object>

In [6]:
#########################################
###### example for plotting lines with px

# # download a zipped shapefile
# wget.download(
#    "https://plotly.github.io/datasets/ne_50m_rivers_lake_centerlines.zip"
# )

# # open a zipped shapefile with the zip:// pseudo-protocol
# geo_df = gpd.read_file("zip://ne_50m_rivers_lake_centerlines.zip")

# lats = []
# lons = []
# names = []

# for feature, name in zip(geo_df.geometry, geo_df.name):
#    if isinstance(feature, shapely.geometry.linestring.LineString):
#        linestrings = [feature]
#    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
#        linestrings = feature.geoms
#    else:
#        continue
#    for linestring in linestrings:
#        x, y = linestring.xy
#        lats = np.append(lats, y)
#        lons = np.append(lons, x)
#        names = np.append(names, [name] * len(y))
#        lats = np.append(lats, None)
#        lons = np.append(lons, None)
#        names = np.append(names, None)

# fig = px.line_geo(
#    lat=lats,
#    lon=lons,
#    hover_name=names,
#    projection="orthographic",
# )

# # add some basic styling to the map
# fig.update_geos(
#    visible=False,
#    resolution=50,
#    showcountries=True,
#    countrycolor="white",
#    showland=True,
#    landcolor="#cccccc",
# )
# fig.show()

<IPython.core.display.Javascript object>

## read buildout grid dfs

In [7]:
########################################################
#### CHOOSE BUILDOUT
#### buildout_dir (contains tables and grid files )
# buildout_name = "ngens89-pen69"
buildout_name = "option2_seed2-ngens11-pen86"
# buildout_name = "option2_seed2-ngens12-pen134"
# ### local
# buildout_dir = f"/Users/isatkaus/projects/ecp/public-github/powerscenarios/data/tables-data/buildouts/{buildout_name}"

### Eagle
buildout_dir = f"/projects/exasgd/isatkaus/buildout/wind/wind_gen_tamu5/buildouts/work-dir/{buildout_name}"


#### if using modified wind gens (new generators) also need
#### new: bus_df, gen_df and wind_gen_df
filename = "bus_df.csv"
bus_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
print("\n bus_df:")
bus_df

filename = "gen_df.csv"
gen_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
print("\n gen_df:")
gen_df

filename = "wind_gen_df.csv"
wind_gen_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
print("\n wind_gen_df:")
wind_gen_df

print("\n branch_df:")
filename = "branch_df.csv"
branch_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
branch_df


 bus_df:


,BusNum,BusName,Latitude,Longitude,Zone
0,1,CREVECOEUR0,40.642116,-89.599560,2
1,2,CREVECOEUR1,40.642116,-89.599560,2
2,3,ILLIOPOLIS0,39.866030,-89.251291,4
3,4,ILLIOPOLIS1,39.866030,-89.251291,4
4,5,PAXTON20,40.378337,-88.105151,6
...,...,...,...,...,...
200,201,NEWWIND201,40.566516,-88.874490,7
201,202,NEWWIND202,40.540800,-89.678600,2
202,203,NEWWIND203,40.470500,-88.398700,6
203,204,NEWWIND204,39.981254,-89.283539,4



 gen_df:


,BusNum,GenID,GenMWMax,GenMWMin,GenWindPowerFactor,GenFuelType,GenUID,BusName,Latitude,Longitude,Zone
0,49,1,4.530000,1.36,1.0,Coal,49_Coal_1,RANTOUL21,40.312222,-88.159444,6
1,50,1,4.530000,1.36,1.0,Coal,50_Coal_1,RANTOUL22,40.312222,-88.159444,6
2,51,1,4.530000,1.36,1.0,Coal,51_Coal_1,RANTOUL23,40.312222,-88.159444,6
3,52,1,4.530000,1.36,1.0,Coal,52_Coal_1,RANTOUL24,40.312222,-88.159444,6
4,53,1,9.070000,2.72,1.0,Coal,53_Coal_1,RANTOUL25,40.312222,-88.159444,6
...,...,...,...,...,...,...,...,...,...,...,...
49,201,1,116.800951,0.00,1.0,Wind,201_Wind_1,NEWWIND201,40.566516,-88.874490,7
50,202,1,26.518775,0.00,1.0,Wind,202_Wind_1,NEWWIND202,40.540800,-89.678600,2
51,203,1,180.146889,0.00,1.0,Wind,203_Wind_1,NEWWIND203,40.470500,-88.398700,6
52,204,1,280.672452,0.00,1.0,Wind,204_Wind_1,NEWWIND204,39.981254,-89.283539,4



 wind_gen_df:


,vor_idx,BusNum,GenID,GenMWMax,GenMWMin,GenWindPowerFactor,GenFuelType,GenUID,BusName,Latitude,Longitude,Zone,incl_generation_mwh
0,43,195,1,246.694011,0.0,1.0,Wind,195_Wind_1,NEWWIND195,40.120833,-88.961111,7,9.734043e+05
1,98,196,1,393.682205,0.0,1.0,Wind,196_Wind_1,NEWWIND196,39.960820,-88.434724,5,1.523061e+06
2,101,197,1,137.268369,0.0,1.0,Wind,197_Wind_1,NEWWIND197,40.171900,-88.833900,7,5.385635e+05
3,76,198,1,96.669832,0.0,1.0,Wind,198_Wind_1,NEWWIND198,40.136481,-88.305307,5,3.822558e+05
4,13,199,1,105.396741,0.0,1.0,Wind,199_Wind_1,NEWWIND199,39.827583,-89.047800,4,4.081816e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,52,201,1,116.800951,0.0,1.0,Wind,201_Wind_1,NEWWIND201,40.566516,-88.874490,7,4.702959e+05
7,69,202,1,26.518775,0.0,1.0,Wind,202_Wind_1,NEWWIND202,40.540800,-89.678600,2,9.544040e+04
8,107,203,1,180.146889,0.0,1.0,Wind,203_Wind_1,NEWWIND203,40.470500,-88.398700,6,7.289372e+05
9,25,204,1,280.672452,0.0,1.0,Wind,204_Wind_1,NEWWIND204,39.981254,-89.283539,4,1.082313e+06



 branch_df:


,FromBus,ToBus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax,Pf,Qf,Pt,Qt,mu_Sf,mu_St,mu_angmin,mu_angmax,distance,from_latitude,from_longitude,to_latitude,to_longitude,trafo
UID,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2,1,0.000673,0.003339,0.00000,100.000000,0.0,0.0,0,0.0,1,0.0,0.0,-7.39,-2.10,7.39,2.11,0.0,0.0,0.0,0.0,0.000000,40.642116,-89.599560,40.642116,-89.599560,True
1,1,114,0.018542,0.119758,0.02285,221.100000,0.0,0.0,0,0.0,1,0.0,0.0,12.25,-0.57,-12.22,-1.63,0.0,0.0,0.0,0.0,32.807597,40.642116,-89.599560,40.870710,-89.353494,False
2,119,1,0.005615,0.036268,0.00692,221.100000,0.0,0.0,0,0.0,1,0.0,0.0,40.13,7.94,-40.04,-8.08,0.0,0.0,0.0,0.0,7.437729,40.595800,-89.663100,40.642116,-89.599560,False
3,187,1,0.004258,0.027501,0.00525,221.100000,0.0,0.0,0,0.0,1,0.0,0.0,-20.39,-6.96,20.41,6.54,0.0,0.0,0.0,0.0,4.773789,40.676541,-89.633346,40.642116,-89.599560,False
4,4,3,0.000573,0.003303,0.00000,100.000000,0.0,0.0,0,0.0,1,0.0,0.0,-1.70,-0.48,1.70,0.48,0.0,0.0,0.0,0.0,0.000000,39.866030,-89.251291,39.866030,-89.251291,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,201,103,0.065683,0.000000,0.00000,152.258345,0.0,0.0,1,0.0,1,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,40.566516,-88.874490,40.566516,-88.874490,True
247,202,128,0.026446,0.000000,0.00000,35.153941,0.0,0.0,1,0.0,1,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,40.540800,-89.678600,40.540800,-89.678600,True
248,203,189,0.057834,0.000000,0.00000,236.456675,0.0,0.0,1,0.0,1,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,40.470500,-88.398700,40.470500,-88.398700,True


<IPython.core.display.Javascript object>

## read original grid dfs

In [8]:
########################################################
#### CHOOSE BUILDOUT: ORIGINAL GRID
#### buildout_dir (contains tables and grid files )
# buildout_name = "ngens89-pen69"
orig_buildout_name = "original-grid-buildout"
# buildout_name = "option2_seed2-ngens12-pen134"
# ### local
# buildout_dir = f"/Users/isatkaus/projects/ecp/public-github/powerscenarios/data/tables-data/buildouts/{buildout_name}"

### Eagle
buildout_dir = f"/projects/exasgd/isatkaus/buildout/wind/wind_gen_tamu5/buildouts/work-dir/{orig_buildout_name}"


#### if using modified wind gens (new generators) also need
#### new: bus_df, gen_df and wind_gen_df
filename = "bus_df.csv"
orig_bus_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
print("\n orig_bus_df:")
orig_bus_df

filename = "gen_df.csv"
orig_gen_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
print("\n orig_gen_df:")
orig_gen_df

filename = "wind_gen_df.csv"
orig_wind_gen_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
print("\n orig_wind_gen_df:")
orig_wind_gen_df


filename = "branch_df.csv"
orig_branch_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
print("\n orig_branch_df:")
orig_branch_df


 orig_bus_df:


,BusNum,BusName,Latitude,Longitude,Zone
0,1,CREVECOEUR0,40.642116,-89.599560,2
1,2,CREVECOEUR1,40.642116,-89.599560,2
2,3,ILLIOPOLIS0,39.866030,-89.251291,4
3,4,ILLIOPOLIS1,39.866030,-89.251291,4
4,5,PAXTON20,40.378337,-88.105151,6
...,...,...,...,...,...
195,196,GIBSONCITY11,40.470500,-88.398700,6
196,197,GIBSONCITY12,40.470500,-88.398700,6
197,198,MAPLETON0,40.556403,-89.762073,2
198,199,GIFFORD0,40.306027,-87.958606,6



 orig_gen_df:


,BusNum,GenID,GenMWMax,GenMWMin,GenWindPowerFactor,GenFuelType,GenUID,BusName,Latitude,Longitude,Zone
0,49,1,4.530000,1.360000,1.0,Coal,49_Coal_1,RANTOUL21,40.312222,-88.159444,6
1,50,1,4.530000,1.360000,1.0,Coal,50_Coal_1,RANTOUL22,40.312222,-88.159444,6
2,51,1,4.530000,1.360000,1.0,Coal,51_Coal_1,RANTOUL23,40.312222,-88.159444,6
3,52,1,4.530000,1.360000,1.0,Coal,52_Coal_1,RANTOUL24,40.312222,-88.159444,6
4,53,1,9.070000,2.720000,1.0,Coal,53_Coal_1,RANTOUL25,40.312222,-88.159444,6
...,...,...,...,...,...,...,...,...,...,...,...
44,182,1,17.500000,5.250000,1.0,Coal,182_Coal_1,SPRINGFIELD21,39.806868,-89.641653,4
45,183,1,26.600000,7.980000,1.0,Coal,183_Coal_1,SPRINGFIELD22,39.806868,-89.641653,4
46,189,1,569.150019,170.749998,1.0,Nuclear,189_Nuclear_1,CLINTON12,40.171900,-88.833900,7
47,196,1,67.500001,20.250000,1.0,NaturalGas,196_NaturalGas_1,GIBSONCITY11,40.470500,-88.398700,6



 orig_wind_gen_df:


,BusNum,GenID,GenMWMax,GenMWMin,GenWindPowerFactor,GenFuelType,GenUID,BusName,Latitude,Longitude,Zone
0,65,1,150.399995,45.120001,1.0,Wind,65_Wind_1,PAXTON11,40.46405,-88.021517,6
1,104,1,99.000001,29.699999,1.0,Wind,104_Wind_1,ELLSWORTH12,40.47920,-88.798900,7
2,105,1,198.000002,59.400004,1.0,Wind,105_Wind_1,ELLSWORTH13,40.47920,-88.798900,7
3,114,1,1.700000,0.510000,1.0,Wind,114_Wind_1,NORMAL22,40.53700,-89.019000,7
4,115,1,150.000000,44.999999,1.0,Wind,115_Wind_1,NORMAL23,40.53700,-89.019000,7
5,147,1,100.500000,30.149999,1.0,Wind,147_Wind_1,HOPEDALE21,40.36920,-89.402200,7



 orig_branch_df:


,FromBus,ToBus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax,Pf,Qf,Pt,Qt,mu_Sf,mu_St,mu_angmin,mu_angmax,distance,from_latitude,from_longitude,to_latitude,to_longitude,trafo
UID,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2,1,0.000673,0.003339,0.00000,100.0,0,0,0,0,1,0,0,-7.39,-2.10,7.39,2.11,0.0,0.0,0.0,0.0,0.000000,40.642116,-89.599560,40.642116,-89.599560,True
1,1,119,0.018542,0.119758,0.02285,221.1,0,0,0,0,1,0,0,12.25,-0.57,-12.22,-1.63,0.0,0.0,0.0,0.0,32.807597,40.642116,-89.599560,40.870710,-89.353494,False
2,124,1,0.005615,0.036268,0.00692,221.1,0,0,0,0,1,0,0,40.13,7.94,-40.04,-8.08,0.0,0.0,0.0,0.0,7.437729,40.595800,-89.663100,40.642116,-89.599560,False
3,193,1,0.004258,0.027501,0.00525,221.1,0,0,0,0,1,0,0,-20.39,-6.96,20.41,6.54,0.0,0.0,0.0,0.0,4.773789,40.676541,-89.633346,40.642116,-89.599560,False
4,4,3,0.000573,0.003303,0.00000,100.0,0,0,0,0,1,0,0,-1.70,-0.48,1.70,0.48,0.0,0.0,0.0,0.0,0.000000,39.866030,-89.251291,39.866030,-89.251291,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,181,194,0.004395,0.028384,0.00542,221.1,0,0,0,0,1,0,0,-35.40,-19.04,35.47,18.88,0.0,0.0,0.0,0.0,5.042087,39.806868,-89.641653,39.762449,-89.629884,False
241,188,187,0.001394,0.044762,0.00000,400.0,0,0,1,0,1,0,0,-113.10,8.78,113.27,-3.48,0.0,0.0,0.0,0.0,0.000000,40.171900,-88.833900,40.171900,-88.833900,True
242,189,187,0.000300,0.007815,0.00000,740.0,0,0,1,0,1,0,0,384.37,-24.15,-383.96,34.87,0.0,0.0,0.0,0.0,0.000000,40.171900,-88.833900,40.171900,-88.833900,True


<IPython.core.display.Javascript object>

In [9]:
# filename = "points_gdf.csv"
# df = pd.read_csv(os.path.join(buildout_dir,filename),index_col =0)
# df
# df['geometry'] = df['geometry'].apply(wkt.loads)
# #### lat/lon
# gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="epsg:4326")
# #### Mercator (I usually save in this crs)
# #gdf = gpd.GeoDataFrame(df, crs='epsg:3857')
# gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="epsg:3857")
# #gdf = gdf.to_crs(epsg=3857)
# gdf

<IPython.core.display.Javascript object>

In [10]:

# fig = px.scatter_geo(bus_df,
#                     lat="Latitude",
#                     lon="Longitude",
#                     hover_name="BusNum")

# fig.update_layout(
#         title = 'Plot with scatter_geo - not as good as scatter_mapbox',
#         geo_scope='usa',
#     )




<IPython.core.display.Javascript object>

## mapbox tiles
* px.scatter_mapbox() works much better than px.scatter_geo() especially on touch devices

In [11]:
bus_df

,BusNum,BusName,Latitude,Longitude,Zone
0,1,CREVECOEUR0,40.642116,-89.599560,2
1,2,CREVECOEUR1,40.642116,-89.599560,2
2,3,ILLIOPOLIS0,39.866030,-89.251291,4
3,4,ILLIOPOLIS1,39.866030,-89.251291,4
4,5,PAXTON20,40.378337,-88.105151,6
...,...,...,...,...,...
200,201,NEWWIND201,40.566516,-88.874490,7
201,202,NEWWIND202,40.540800,-89.678600,2
202,203,NEWWIND203,40.470500,-88.398700,6
203,204,NEWWIND204,39.981254,-89.283539,4


<IPython.core.display.Javascript object>

In [12]:
fig = px.scatter_mapbox(
    bus_df,
    lat="Latitude",
    lon="Longitude",
    hover_name="BusName",
    hover_data=["BusNum", "Zone"],
    color_discrete_sequence=["fuchsia"],
    zoom=4,
    height=300,
    title=f"buses: {buildout_name}",
)
fig.update_layout(
    mapbox_style="open-street-map", margin={"r": 0, "t": 40, "l": 0, "b": 0}
)
# fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

<IPython.core.display.Javascript object>

In [13]:
fig2 = px.scatter_mapbox(
    wind_gen_df,
    lat="Latitude",
    lon="Longitude",
    hover_name="BusNum",
    hover_data=["GenMWMax", "BusName"],
    color_discrete_sequence=["blue"],
    zoom=4,
    height=300,
)

fig2.update_layout(
    mapbox_style="open-street-map", margin={"r": 0, "t": 0, "l": 0, "b": 0}
)

<IPython.core.display.Javascript object>

In [14]:
##################################################################
#### two dataframes in one plot: with fig1.add_trace(fig2.data[0])


fig1 = px.scatter_mapbox(
    bus_df,
    lat="Latitude",
    lon="Longitude",
    hover_name="BusName",
    hover_data=["BusNum", "Zone"],
    color_discrete_sequence=["fuchsia"],
    zoom=4,
    height=300,
)

fig2 = px.scatter_mapbox(
    wind_gen_df,
    lat="Latitude",
    lon="Longitude",
    hover_name="BusNum",
    hover_data=["GenMWMax", "BusName"],
    color_discrete_sequence=["blue"],
    zoom=4,
    height=300,
)


_ = fig1.add_trace(fig2.data[0])


fig1.update_layout(
    mapbox_style="open-street-map", margin={"r": 0, "t": 0, "l": 0, "b": 0}
)

<IPython.core.display.Javascript object>

In [15]:
branch_df

,FromBus,ToBus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax,Pf,Qf,Pt,Qt,mu_Sf,mu_St,mu_angmin,mu_angmax,distance,from_latitude,from_longitude,to_latitude,to_longitude,trafo
UID,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2,1,0.000673,0.003339,0.00000,100.000000,0.0,0.0,0,0.0,1,0.0,0.0,-7.39,-2.10,7.39,2.11,0.0,0.0,0.0,0.0,0.000000,40.642116,-89.599560,40.642116,-89.599560,True
1,1,114,0.018542,0.119758,0.02285,221.100000,0.0,0.0,0,0.0,1,0.0,0.0,12.25,-0.57,-12.22,-1.63,0.0,0.0,0.0,0.0,32.807597,40.642116,-89.599560,40.870710,-89.353494,False
2,119,1,0.005615,0.036268,0.00692,221.100000,0.0,0.0,0,0.0,1,0.0,0.0,40.13,7.94,-40.04,-8.08,0.0,0.0,0.0,0.0,7.437729,40.595800,-89.663100,40.642116,-89.599560,False
3,187,1,0.004258,0.027501,0.00525,221.100000,0.0,0.0,0,0.0,1,0.0,0.0,-20.39,-6.96,20.41,6.54,0.0,0.0,0.0,0.0,4.773789,40.676541,-89.633346,40.642116,-89.599560,False
4,4,3,0.000573,0.003303,0.00000,100.000000,0.0,0.0,0,0.0,1,0.0,0.0,-1.70,-0.48,1.70,0.48,0.0,0.0,0.0,0.0,0.000000,39.866030,-89.251291,39.866030,-89.251291,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,201,103,0.065683,0.000000,0.00000,152.258345,0.0,0.0,1,0.0,1,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,40.566516,-88.874490,40.566516,-88.874490,True
247,202,128,0.026446,0.000000,0.00000,35.153941,0.0,0.0,1,0.0,1,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,40.540800,-89.678600,40.540800,-89.678600,True
248,203,189,0.057834,0.000000,0.00000,236.456675,0.0,0.0,1,0.0,1,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,40.470500,-88.398700,40.470500,-88.398700,True


<IPython.core.display.Javascript object>

In [16]:
branch_df.loc[branch_df["FromBus"] == 193]

,FromBus,ToBus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax,Pf,Qf,Pt,Qt,mu_Sf,mu_St,mu_angmin,mu_angmax,distance,from_latitude,from_longitude,to_latitude,to_longitude,trafo
UID,,,,,,,,,,,,,,,,,,,,,,,,,,,
40,193,25,0.009909,0.064003,0.01221,221.1,0.0,0.0,0,0.0,1,0.0,0.0,-49.33,9.96,49.57,-9.8,0.0,0.0,0.0,0.0,15.865363,40.306027,-87.958606,40.438957,-87.890723,False
225,193,166,0.024005,0.155042,0.02958,221.1,0.0,0.0,0,0.0,1,0.0,0.0,12.12,-3.91,-12.09,0.9,0.0,0.0,0.0,0.0,43.528938,40.306027,-87.958606,39.916122,-87.914791,False


<IPython.core.display.Javascript object>

In [34]:
branch_df

,ToBus,FromBus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax,Pf,Qf,Pt,Qt,mu_Sf,mu_St,mu_angmin,mu_angmax,distance,from_latitude,from_longitude,to_latitude,to_longitude,trafo
UID,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,2,0.000673,0.003339,0.00000,100.0,0,0,0,0,1,0,0,-7.39,-2.10,7.39,2.11,0.0,0.0,0.0,0.0,0.000000,40.642116,-89.601560,40.642116,-89.597560,True
1,119,1,0.018542,0.119758,0.02285,221.1,0,0,0,0,1,0,0,12.25,-0.57,-12.22,-1.63,0.0,0.0,0.0,0.0,32.807597,40.642116,-89.597560,40.870710,-89.351494,False
2,1,124,0.005615,0.036268,0.00692,221.1,0,0,0,0,1,0,0,40.13,7.94,-40.04,-8.08,0.0,0.0,0.0,0.0,7.437729,40.597702,-89.662482,40.642116,-89.597560,False
3,1,193,0.004258,0.027501,0.00525,221.1,0,0,0,0,1,0,0,-20.39,-6.96,20.41,6.54,0.0,0.0,0.0,0.0,4.773789,40.676541,-89.631346,40.642116,-89.597560,False
4,3,4,0.000573,0.003303,0.00000,100.0,0,0,0,0,1,0,0,-1.70,-0.48,1.70,0.48,0.0,0.0,0.0,0.0,0.000000,39.866030,-89.253291,39.866030,-89.249291,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,194,181,0.004395,0.028384,0.00542,221.1,0,0,0,0,1,0,0,-35.40,-19.04,35.47,18.88,0.0,0.0,0.0,0.0,5.042087,39.806868,-89.639653,39.762449,-89.627884,False
241,187,188,0.001394,0.044762,0.00000,400.0,0,0,1,0,1,0,0,-113.10,8.78,113.27,-3.48,0.0,0.0,0.0,0.0,0.000000,40.173632,-88.834900,40.171900,-88.831900,True
242,187,189,0.000300,0.007815,0.00000,740.0,0,0,1,0,1,0,0,384.37,-24.15,-383.96,34.87,0.0,0.0,0.0,0.0,0.000000,40.170168,-88.834900,40.171900,-88.831900,True


<IPython.core.display.Javascript object>

In [56]:
########################################
##### branch plotting: plotly.geo_objects.ScatterGeo
##### works but can't combine figures with plotly.express?
lats = []
lons = []
names = []

for uid in branch_df.index[:]:
    #uid = 1
    from_lat = branch_df.loc[uid,"from_latitude"]
    to_lat = branch_df.loc[uid,"to_latitude"]
    from_lon = branch_df.loc[uid,"from_longitude"]
    to_lon = branch_df.loc[uid,"to_longitude"]
    lats = np.append(lats, from_lat)
    lats = np.append(lats,to_lat)
    names = np.append(names,uid)
    lons = np.append(lons,from_lon)
    lons = np.append(lons,to_lon)
    names = np.append(names,uid)
    lats = np.append(lats,None)
    lons = np.append(lons,None)
    names = np.append(names,None)

# lats
# lons

fig =go.Figure()

fig3 = go.Scattergeo(
    locationmode="USA-states",
    lon=lons,
    lat=lats,
    mode="lines",
    line=dict(width=1, color="red"),
    opacity=1,

)



fig.add_trace(fig3)






<IPython.core.display.Javascript object>

In [82]:
########################################
##### branch plotting: plotly.express.line_geo
##### works but can't combine figures with plotly.express?
lats = []
lons = []
names = []

for uid in branch_df.index[:]:
    #uid = 1
    from_lat = branch_df.loc[uid,"from_latitude"]
    to_lat = branch_df.loc[uid,"to_latitude"]
    from_lon = branch_df.loc[uid,"from_longitude"]
    to_lon = branch_df.loc[uid,"to_longitude"]
    lats = np.append(lats, from_lat)
    lats = np.append(lats,to_lat)
    names = np.append(names,uid)
    lons = np.append(lons,from_lon)
    lons = np.append(lons,to_lon)
    names = np.append(names,uid)
    lats = np.append(lats,None)
    lons = np.append(lons,None)
    names = np.append(names,None)

# lats
# lons

#fig3 =go.Figure()

fig = px.line_geo(
    
    lon=lons,
    lat=lats,
    locationmode="USA-states",
    #mode="lines",
    #line=dict(width=1, color="red"),
    #opacity=1,
    hover_name=names,
    scope = "usa",
    basemap_visible=True,
)

fig.show()

#fig3.add_trace(fig.data[0])

### does not work with line_geo (works with line_mapbox)
# fig3.update_layout(
#     mapbox_style="open-street-map", margin={"r": 0, "t": 0, "l": 0, "b": 0}
# )

# ### USGS custom raster files (does not work for line_geo)
# fig3.update_layout(
#     mapbox_style="white-bg",
#     mapbox_layers=[
#         {
#             "below": "traces",
#             "sourcetype": "raster",
#             "sourceattribution": "United States Geological Survey",
#             "source": [
#                 "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
#             ],
#         }
#     ],
# )




#fig.show()




<IPython.core.display.Javascript object>

In [68]:
?px.line_geo

<IPython.core.display.Javascript object>

In [18]:
from_lat = branch_df.loc[uid,"from_latitude"]
to_lat = branch_df.loc[uid,"to_latitude"]
from_lon = branch_df.loc[uid,"from_longitude"]
to_lon = branch_df.loc[uid,"to_longitude"]
lats = [from_lat, to_lat]
lons = [from_lon, to_lon]


# fig = px.line_geo(lat=lats, lon=lons,)
# fig
fig = px.line_mapbox(lat=lats, lon=lons,)

fig2 = px.scatter_mapbox(
    bus_df,
    lat="Latitude",
    lon="Longitude",
    hover_name="BusNum",
    hover_data=[ "BusName"],
    color_discrete_sequence=["black"],
    zoom=4,
    height=300,
)

#_ = fig.add_trace(fig2.data[0])

_ = fig2.add_trace(fig.data[0])


fig2.update_layout(
    mapbox_style="open-street-map", margin={"r": 0, "t": 0, "l": 0, "b": 0}
)


<IPython.core.display.Javascript object>

In [19]:
df = branch_df.head(60).copy()
df

,FromBus,ToBus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax,Pf,Qf,Pt,Qt,mu_Sf,mu_St,mu_angmin,mu_angmax,distance,from_latitude,from_longitude,to_latitude,to_longitude,trafo
UID,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2,1,0.000673,0.003339,0.00000,100.0,0.0,0.0,0,0.0,1,0.0,0.0,-7.39,-2.10,7.39,2.11,0.0,0.0,0.0,0.0,0.000000,40.642116,-89.599560,40.642116,-89.599560,True
1,1,114,0.018542,0.119758,0.02285,221.1,0.0,0.0,0,0.0,1,0.0,0.0,12.25,-0.57,-12.22,-1.63,0.0,0.0,0.0,0.0,32.807597,40.642116,-89.599560,40.870710,-89.353494,False
2,119,1,0.005615,0.036268,0.00692,221.1,0.0,0.0,0,0.0,1,0.0,0.0,40.13,7.94,-40.04,-8.08,0.0,0.0,0.0,0.0,7.437729,40.595800,-89.663100,40.642116,-89.599560,False
3,187,1,0.004258,0.027501,0.00525,221.1,0.0,0.0,0,0.0,1,0.0,0.0,-20.39,-6.96,20.41,6.54,0.0,0.0,0.0,0.0,4.773789,40.676541,-89.633346,40.642116,-89.599560,False
4,4,3,0.000573,0.003303,0.00000,100.0,0.0,0.0,0,0.0,1,0.0,0.0,-1.70,-0.48,1.70,0.48,0.0,0.0,0.0,0.0,0.000000,39.866030,-89.251291,39.866030,-89.251291,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,38,36,0.000670,0.003511,0.00000,100.0,0.0,0.0,0,0.0,1,0.0,0.0,-29.81,-8.50,29.81,8.53,0.0,0.0,0.0,0.0,0.000000,40.214556,-89.299252,40.214556,-89.299252,True
56,58,36,0.010889,0.074390,0.01266,221.1,0.0,0.0,0,0.0,1,0.0,0.0,4.99,8.16,-4.98,-9.43,0.0,0.0,0.0,0.0,17.788889,40.232881,-89.091171,40.214556,-89.299252,False
57,82,36,0.009615,0.066986,0.01095,221.1,0.0,0.0,0,0.0,1,0.0,0.0,28.47,-0.53,-28.40,-0.11,0.0,0.0,0.0,0.0,15.288501,40.350757,-89.323483,40.214556,-89.299252,False


<IPython.core.display.Javascript object>

In [20]:
# a = df["from_latitude"].to_numpy()
# b = df["to_latitude"].to_numpy()
# x = np.empty(29)
# x[::3] = a
# x

<IPython.core.display.Javascript object>

In [21]:
from_lats = df["from_latitude"].tolist()
to_lats = df["to_latitude"].tolist()
from_lons = df["from_longitude"].tolist()
to_lons = df["to_longitude"].tolist()

lats = [None for k in range(3 * len(df) - 1)]
lats[::3] = from_lats
lats[1::3] = to_lats
lats
lons = [None for k in range(3 * len(df) - 1)]
lons[::3] = from_lats
lons[1::3] = to_lats
lons
# # l[::2] = fl
# # l
# lats = [a for b in zip(from_lats, to_lats) for a in b]
# lons = [a for b in zip(from_lons, to_lons) for a in b]
# lats
# lons


# # list(zip(from_lats, to_lats))

# # df["from_longitude"]
# # df["to_longitude"]

[40.642116,
 40.642116,
 None,
 40.642116,
 40.87071,
 None,
 40.5958,
 40.642116,
 None,
 40.676541,
 40.642116,
 None,
 39.86603,
 39.86603,
 None,
 39.981254,
 39.86603,
 None,
 39.86603,
 39.855374,
 None,
 40.378337,
 40.378337,
 None,
 40.312222,
 40.378337,
 None,
 40.378337,
 40.46405,
 None,
 40.712882,
 40.712882,
 None,
 40.712882,
 40.746726,
 None,
 40.712882,
 40.697618,
 None,
 40.712882,
 40.704146,
 None,
 40.705729,
 40.705729,
 None,
 40.705729,
 40.5958,
 None,
 40.737893,
 40.705729,
 None,
 40.705729,
 40.674568,
 None,
 40.705729,
 40.676541,
 None,
 39.801351,
 39.801351,
 None,
 39.801351,
 39.801351,
 None,
 39.801351,
 39.81707,
 None,
 39.801351,
 39.798175,
 None,
 39.719105,
 39.801351,
 None,
 39.81707,
 39.81707,
 None,
 39.81707,
 40.200639,
 None,
 39.81707,
 39.757327,
 None,
 39.81707,
 39.81707,
 None,
 40.17288,
 40.17288,
 None,
 40.17288,
 40.17288,
 None,
 40.17288,
 40.17288,
 None,
 40.17288,
 40.17288,
 None,
 40.296711,
 40.17288,
 None,
 40

[40.642116,
 40.642116,
 None,
 40.642116,
 40.87071,
 None,
 40.5958,
 40.642116,
 None,
 40.676541,
 40.642116,
 None,
 39.86603,
 39.86603,
 None,
 39.981254,
 39.86603,
 None,
 39.86603,
 39.855374,
 None,
 40.378337,
 40.378337,
 None,
 40.312222,
 40.378337,
 None,
 40.378337,
 40.46405,
 None,
 40.712882,
 40.712882,
 None,
 40.712882,
 40.746726,
 None,
 40.712882,
 40.697618,
 None,
 40.712882,
 40.704146,
 None,
 40.705729,
 40.705729,
 None,
 40.705729,
 40.5958,
 None,
 40.737893,
 40.705729,
 None,
 40.705729,
 40.674568,
 None,
 40.705729,
 40.676541,
 None,
 39.801351,
 39.801351,
 None,
 39.801351,
 39.801351,
 None,
 39.801351,
 39.81707,
 None,
 39.801351,
 39.798175,
 None,
 39.719105,
 39.801351,
 None,
 39.81707,
 39.81707,
 None,
 39.81707,
 40.200639,
 None,
 39.81707,
 39.757327,
 None,
 39.81707,
 39.81707,
 None,
 40.17288,
 40.17288,
 None,
 40.17288,
 40.17288,
 None,
 40.17288,
 40.17288,
 None,
 40.17288,
 40.17288,
 None,
 40.296711,
 40.17288,
 None,
 40

<IPython.core.display.Javascript object>

In [22]:
# fig = px.line_mapbox(
#     data_frame=df,
#     lat=["from_latitude","to_latitude"],
#     lon=["from_longitude","from_longitude"]
# )


# fig = px.line_mapbox(
#     #data_frame=df,
#     lat=lats,
#     lon=lons,
# )



fig2 = px.scatter_mapbox(
    bus_df,
    lat="Latitude",
    lon="Longitude",
    hover_name="BusNum",
    hover_data=[ "BusName"],
    color_discrete_sequence=["black"],
    zoom=4,
    height=800,
)

#_ = fig.add_trace(fig2.data[0])



# fig = px.line_mapbox(
#     #data_frame=df,
#     lat=from_lats,
#     lon=from_lons,
# )

for uid in branch_df.index[:]:
    
    from_lat = branch_df.loc[uid,"from_latitude"]
    to_lat = branch_df.loc[uid,"to_latitude"]
    from_lon = branch_df.loc[uid,"from_longitude"]
    to_lon = branch_df.loc[uid,"to_longitude"]
    
    fig = px.line_mapbox(
    #data_frame=df,
    lat=[from_lat,to_lat],
    lon=[from_lon,to_lon],
    #hover_name = uid,
    )
    _ = fig2.add_trace(fig.data[0])
    


#_ = fig2.add_trace(fig.data[0])


fig2.update_layout(
    mapbox_style="open-street-map", margin={"r": 0, "t": 0, "l": 0, "b": 0}
)


<IPython.core.display.Javascript object>

In [23]:
?px.line_mapbox

<IPython.core.display.Javascript object>

## splitpoints
* split buses that are in the same location (same lat/lon)
* split: place them in a circle of radius R spaced by equal theta=360/n_colocated_buses

In [24]:
# lat = 40.4792
# lon = -88.7989
# bus_df.loc[(bus_df["Latitude"] == lat) & (bus_df["Longitude"] == lon)]

# bus_df.duplicated(["Latitude", "Longitude"])

<IPython.core.display.Javascript object>

In [25]:
# %%capture
# filename = "bus_df.csv"
# bus_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
# ### unique locations (will be centers of our split points stars)
# loc_df = (
#     bus_df[["Latitude", "Longitude"]]
#     .groupby(["Latitude", "Longitude"])
#     .first()
#     .reset_index()
# )
# # loc_df
# ## radius
# R = 0.002

# for idx in loc_df.index[:]:
#     lat = loc_df.loc[idx, "Latitude"]
#     lon = loc_df.loc[idx, "Longitude"]
#     df = bus_df.loc[(bus_df["Latitude"] == lat) & (bus_df["Longitude"] == lon)]
#     n_buses = len(df)
#     print("\n colocated buses:")
#     df
#     ### center of the circle/star
#     x0 = lon
#     y0 = lat
#     print(f"\n center of star: ({lat},{lon})")
#     theta = 2 * np.pi / n_buses
#     print(f"\n theta: {theta}")
#     ### change coordinates (lat/lon) of colocated points
#     for k, bus_idx in enumerate(df.index):
#         print(f"\n bus_idx: {bus_idx}")
#         print(f"\n k: {k}")
#         x_new = x0 + R * np.cos(k * theta)
#         y_new = y0 + R * np.sin(k * theta)
#         ### overwrite lat/lons in bus_df
#         bus_df.loc[bus_idx, "Latitude"] = y_new
#         bus_df.loc[bus_idx, "Longitude"] = x_new


# # bus_df[["Latitude", "Longitude"]].size().reset_index().rename(columns={0: "count"})

<IPython.core.display.Javascript object>

In [26]:
def split_points(bus_df):
    
    """ 
    modifies bus_df lot/lons to split colocated points
    """

    ### unique locations (will be centers of our split points stars)
    loc_df = (
        bus_df[["Latitude", "Longitude"]]
        .groupby(["Latitude", "Longitude"])
        .first()
        .reset_index()
    )
    # loc_df
    ## radius
    R = 0.002

    for idx in loc_df.index[:]:
        lat = loc_df.loc[idx, "Latitude"]
        lon = loc_df.loc[idx, "Longitude"]
        df = bus_df.loc[(bus_df["Latitude"] == lat) & (bus_df["Longitude"] == lon)]
        n_buses = len(df)
        print("\n colocated buses:")
        df
        ### center of the circle/star
        x0 = lon
        y0 = lat
        print(f"\n center of star: ({lat},{lon})")
        theta = 2 * np.pi / n_buses
        print(f"\n theta: {theta}")
        ### change coordinates (lat/lon) of colocated points
        for k, bus_idx in enumerate(df.index):
            print(f"\n bus_idx: {bus_idx}")
            print(f"\n k: {k}")
            x_new = x0 + R * np.cos(k * theta)
            y_new = y0 + R * np.sin(k * theta)
            ### overwrite lat/lons in bus_df
            bus_df.loc[bus_idx, "Latitude"] = y_new
            bus_df.loc[bus_idx, "Longitude"] = x_new
            
    return bus_df


<IPython.core.display.Javascript object>

In [27]:
%%capture
filename = "bus_df.csv"
bus_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
bus_df = split_points(bus_df)

<IPython.core.display.Javascript object>

In [29]:
fig = px.scatter_mapbox(
    bus_df,
    lat="Latitude",
    lon="Longitude",
    hover_name="BusName",
    hover_data=["BusNum", "Zone"],
    color_discrete_sequence=["fuchsia"],
    zoom=4,
    height=300,
)
fig.update_layout(
    mapbox_style="open-street-map", margin={"r": 0, "t": 0, "l": 0, "b": 0}
)

<IPython.core.display.Javascript object>

## modify lat/lons in other df's
* gen_df
* branch_df


In [30]:
################################################
#### replace lat/lons in other dfs
filename = "gen_df.csv"
gen_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)

### modify gen_df
bus_idxs = gen_df["BusNum"].values
lats = bus_df.set_index("BusNum").loc[bus_idxs, "Latitude"]
lons = bus_df.set_index("BusNum").loc[bus_idxs, "Longitude"]
df = gen_df.set_index("BusNum")
df["Latitude"] = lats
df["Longitude"] = lons
gen_df = df.reset_index()

################################################
#### branch

filename = "branch_df.csv"
branch_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
# branch_df

### modify branch_df
bus_idxs = branch_df["FromBus"].values
lats = bus_df.set_index("BusNum").loc[bus_idxs, "Latitude"]
lons = bus_df.set_index("BusNum").loc[bus_idxs, "Longitude"]
df = branch_df.set_index("FromBus")
df["from_latitude"] = lats
df["from_longitude"] = lons
branch_df = df.reset_index()

bus_idxs = branch_df["ToBus"].values
lats = bus_df.set_index("BusNum").loc[bus_idxs, "Latitude"]
lons = bus_df.set_index("BusNum").loc[bus_idxs, "Longitude"]
df = branch_df.set_index("ToBus")
df["to_latitude"] = lats
df["to_longitude"] = lons

branch_df = df.reset_index()
branch_df.index.name = "UID"

branch_df

<IPython.core.display.Javascript object>

,ToBus,FromBus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax,Pf,Qf,Pt,Qt,mu_Sf,mu_St,mu_angmin,mu_angmax,distance,from_latitude,from_longitude,to_latitude,to_longitude,trafo
UID,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,2,0.000673,0.003339,0.00000,100.0,0,0,0,0,1,0,0,-7.39,-2.10,7.39,2.11,0.0,0.0,0.0,0.0,0.000000,40.642116,-89.601560,40.642116,-89.597560,True
1,119,1,0.018542,0.119758,0.02285,221.1,0,0,0,0,1,0,0,12.25,-0.57,-12.22,-1.63,0.0,0.0,0.0,0.0,32.807597,40.642116,-89.597560,40.870710,-89.351494,False
2,1,124,0.005615,0.036268,0.00692,221.1,0,0,0,0,1,0,0,40.13,7.94,-40.04,-8.08,0.0,0.0,0.0,0.0,7.437729,40.597702,-89.662482,40.642116,-89.597560,False
3,1,193,0.004258,0.027501,0.00525,221.1,0,0,0,0,1,0,0,-20.39,-6.96,20.41,6.54,0.0,0.0,0.0,0.0,4.773789,40.676541,-89.631346,40.642116,-89.597560,False
4,3,4,0.000573,0.003303,0.00000,100.0,0,0,0,0,1,0,0,-1.70,-0.48,1.70,0.48,0.0,0.0,0.0,0.0,0.000000,39.866030,-89.253291,39.866030,-89.249291,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,194,181,0.004395,0.028384,0.00542,221.1,0,0,0,0,1,0,0,-35.40,-19.04,35.47,18.88,0.0,0.0,0.0,0.0,5.042087,39.806868,-89.639653,39.762449,-89.627884,False
241,187,188,0.001394,0.044762,0.00000,400.0,0,0,1,0,1,0,0,-113.10,8.78,113.27,-3.48,0.0,0.0,0.0,0.0,0.000000,40.173632,-88.834900,40.171900,-88.831900,True
242,187,189,0.000300,0.007815,0.00000,740.0,0,0,1,0,1,0,0,384.37,-24.15,-383.96,34.87,0.0,0.0,0.0,0.0,0.000000,40.170168,-88.834900,40.171900,-88.831900,True


<IPython.core.display.Javascript object>

In [32]:
###################################################
#### plot with branches

fig2 = px.scatter_mapbox(
    bus_df,
    lat="Latitude",
    lon="Longitude",
    hover_name="BusNum",
    hover_data=[ "BusName"],
    color_discrete_sequence=["black"],
    zoom=4,
    height=300,
)

#_ = fig.add_trace(fig2.data[0])



# fig = px.line_mapbox(
#     #data_frame=df,
#     lat=from_lats,
#     lon=from_lons,
# )

for uid in branch_df.index[:]:
    
    from_lat = branch_df.loc[uid,"from_latitude"]
    to_lat = branch_df.loc[uid,"to_latitude"]
    from_lon = branch_df.loc[uid,"from_longitude"]
    to_lon = branch_df.loc[uid,"to_longitude"]
    
    fig = px.line_mapbox(
    #data_frame=df,
    lat=[from_lat,to_lat],
    lon=[from_lon,to_lon],
    #hover_name = uid,
    )
    _ = fig2.add_trace(fig.data[0])
    


#_ = fig2.add_trace(fig.data[0])


fig2.update_layout(
    mapbox_style="open-street-map", margin={"r": 0, "t": 0, "l": 0, "b": 0}
)


<IPython.core.display.Javascript object>

In [308]:
##################################################################
#### two dataframes in one plot: with fig1.add_trace(fig2.data[0])


fig1 = px.scatter_mapbox(
    bus_df,
    lat="Latitude",
    lon="Longitude",
    hover_name="BusName",
    hover_data=["BusNum", "Zone"],
    color_discrete_sequence=["fuchsia"],
    zoom=4,
    height=300,
)

fig2 = px.scatter_mapbox(
    gen_df,
    lat="Latitude",
    lon="Longitude",
    hover_name="GenUID",
    hover_data=[
        "BusName",
        "GenFuelType",
        "GenMWMax",
    ],
    color_discrete_sequence=["blue"],
    zoom=4,
    height=300,
)


_ = fig1.add_trace(fig2.data[0])


fig1.update_layout(
    mapbox_style="open-street-map", margin={"r": 0, "t": 0, "l": 0, "b": 0}
)

<IPython.core.display.Javascript object>

In [309]:
# filename = "selected_gdf.csv"
# selected_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
# selected_df.reset_index(inplace=True)
# selected_df

# fig4 = px.scatter_mapbox(
#     selected_df,
#     lat="latitude",
#     lon="longitude",
#     hover_name="gid",
#     hover_data=["gen_uid","cf_mean", "generation_mwh"],
#     color_discrete_sequence=["fuchsia"],
#     zoom=3,
#     height=600,
# )
# _ = fig4.update_layout(mapbox_style="open-street-map")
# ### add elementrs from another fig
# ### fig has trace and layout (use trace only)
# fig4.add_trace(fig3.data[0])
# fig4.show()

<IPython.core.display.Javascript object>

In [310]:
# filename = "high_cf_incl_df.csv"
# high_cf_incl_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
# high_cf_incl_df.reset_index(inplace=True)
# ### round df numbers so that 16 sig figs don't show in hover tool
# high_cf_incl_df[["incl_generation_mwh", "incl_capacity_mw"]] = round(high_cf_incl_df[["incl_generation_mwh", "incl_capacity_mw"]],2)
# high_cf_incl_df

# fig5 = px.scatter_mapbox(
#     high_cf_incl_df,
#     lat="latitude",
#     lon="longitude",
#     hover_name="gid",
#     hover_data=["cf_mean", "incl_generation_mwh", "incl_capacity_mw"],
#     color_discrete_sequence=["green"],
#     zoom=3,
#     height=600,
# )
# _ = fig5.update_layout(mapbox_style="open-street-map")
# fig5.show()

<IPython.core.display.Javascript object>

## new wind gens

In [311]:
filename = "wind_gen_df.csv"
wind_gen_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
wind_gen_df.reset_index(inplace=True)
wind_gen_df
### round df numbers so that 16 sig figs don't show in hover tool
wind_gen_df[["incl_generation_mwh"]] = round(wind_gen_df[["incl_generation_mwh"]], 2)
wind_gen_df

fig5 = px.scatter_mapbox(
    wind_gen_df,
    lat="Latitude",
    lon="Longitude",
    hover_name="GenUID",
    hover_data=["vor_idx", "incl_generation_mwh"],
    color_discrete_sequence=["green"],
    zoom=3,
    height=600,
)
_ = fig5.update_layout(mapbox_style="open-street-map")
fig5.show()

,index,BusNum,GenID,GenMWMax,GenMWMin,GenWindPowerFactor,GenFuelType,GenUID,BusName,Latitude,Longitude,Zone
0,0,65,1,150.399995,45.120001,1.0,Wind,65_Wind_1,PAXTON11,40.46405,-88.021517,6
1,1,104,1,99.000001,29.699999,1.0,Wind,104_Wind_1,ELLSWORTH12,40.47920,-88.798900,7
2,2,105,1,198.000002,59.400004,1.0,Wind,105_Wind_1,ELLSWORTH13,40.47920,-88.798900,7
3,3,114,1,1.700000,0.510000,1.0,Wind,114_Wind_1,NORMAL22,40.53700,-89.019000,7
4,4,115,1,150.000000,44.999999,1.0,Wind,115_Wind_1,NORMAL23,40.53700,-89.019000,7
5,5,147,1,100.500000,30.149999,1.0,Wind,147_Wind_1,HOPEDALE21,40.36920,-89.402200,7


KeyError: "None of [Index(['incl_generation_mwh'], dtype='object')] are in the [columns]"

<IPython.core.display.Javascript object>

# Scratch

## stack df for animations
* wind_gen_df with wind_df

In [ ]:
###########################################################
#### make smaller wind df

filename = "wind_df.csv"
wind_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
### just take a small piece of entire 7 years
wind_df
#### make small version of wind_df
### 9 months of 7 years
small_wind_df = wind_df.head(12*24*30*9)
filename = "small_wind_df.csv"
small_wind_df.to_csv(os.path.join(buildout_dir, filename))

In [ ]:

filename = "small_wind_df.csv"
wind_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0, parse_dates=True)
wind_df.columns.name="GenUID"
wind_df 


In [ ]:
## just take a small piece of entire 7 years
#wind_df = wind_df.head(20)
start = pd.Timestamp("2007-01-01 00:00:00+00:00")
end = pd.Timestamp("2007-01-01 12:00:00+00:00")
step = pd.Timedelta("20min")
idxs = pd.date_range(start=start, end=end,freq=step)
#wind_df = wind_df.loc[start:step:end]
wind_df = wind_df.loc[idxs]
wind_df

# s = wind_df.T.stack()
# s.name = "power"
# df = s.to_frame().reset_index(level=[0,1]).set_index("GenUID")
# wind_df = pd.merge(wind_gen_df,df,on="GenUID")
# wind_df 

In [ ]:
## just take a small piece of entire 7 years
#wind_df = wind_df.head(20)
start = pd.Timestamp("2007-01-01 00:00:00+00:00")
end = pd.Timestamp("2007-01-01 12:00:00+00:00")
wind_df.loc[start:end]
# s = wind_df.T.stack()
# s.name = "power"
# df = s.to_frame().reset_index(level=[0,1]).set_index("GenUID")
# wind_df = pd.merge(wind_gen_df,df,on="GenUID")
# wind_df 

In [ ]:
######################################################################
#### or animation of timeseries
#### wind_gen_df and wind_df
filename = "wind_gen_df.csv"
wind_gen_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0)
wind_gen_df.reset_index(inplace=True)
### round df numbers so that 16 sig figs don't show in hover tool
wind_gen_df[["incl_generation_mwh"]] = round(wind_gen_df[["incl_generation_mwh"]], 2)
wind_gen_df = wind_gen_df[["vor_idx", "GenMWMax", "GenUID", "Latitude", "Longitude"]]

# wind_df.T
#filename = "wind_df.csv"
### use instead a smaller file
filename = "small_wind_df.csv"
wind_df = pd.read_csv(
    os.path.join(buildout_dir, filename),
    index_col=0,
    parse_dates=True,
)
#wind_df
wind_df.columns.name = "GenUID"
### just take a small piece of available 7 years
start = pd.Timestamp("2007-01-01 00:00:00+00:00")
end = pd.Timestamp("2007-01-02 00:00:00+00:00")
step = pd.Timedelta("20min")
### below 2 lines will return slice, but index name will dissapear
#idxs = pd.date_range(start=start, end=end, freq=step)
#wind_df = wind_df.loc[idxs]
### below line keeps the index name
wind_df = wind_df.loc[start:end].resample(step).first()
#wind_df
# wind_df = wind_df.head(20)
# wind_df
s = wind_df.T.stack()
s.name = "power"
df2 = s.to_frame().reset_index(level=[0, 1]).set_index("GenUID")
wind_df = pd.merge(wind_gen_df, df2, on="GenUID")
wind_df

In [ ]:
wind_df["power"].max()
wind_df["power"].min()

In [ ]:
# wind_df.T
#filename = "wind_df.csv"
### use instead a smaller file
filename = "small_wind_df.csv"
wind_df = pd.read_csv(
    os.path.join(buildout_dir, filename),
    index_col=0,
    parse_dates=True,
)
wind_df.columns.name = "GenUID"
wind_df
# ### just take a small piece of available 7 years
# start = pd.Timestamp("2007-01-01 00:00:00+00:00")
# end = pd.Timestamp("2007-01-02 00:00:00+00:00")
# step = pd.Timedelta("20min")
# idxs = pd.date_range(start=start, end=end, freq=step)
# # wind_df = wind_df.loc[start:step:end]
# wind_df = wind_df.loc[idxs]
# wind_df

In [ ]:
start = pd.Timestamp("2007-01-01 00:00:00+00:00")
end = pd.Timestamp("2007-01-02 00:00:00+00:00")
step = pd.Timedelta("20min")
idxs = pd.date_range(start=start, end=end, freq=step)
# wind_df = wind_df.loc[start:step:end]
wind_df.loc[idxs]
wind_df.head()


## interesting ramps in wind_df
* Feb 7-11

In [ ]:
wind_df = pd.read_csv(os.path.join(buildout_dir, filename), index_col=0, parse_dates=True)
wind_df.columns.name="GenUID"
wind_df 

In [ ]:
# range_selector = {'steps':['1y','2 months','5 weeks','ytd','2mtd'],
#                                         'axis':'xaxis', 'bgcolor' : ('blue',.3),
#                                         'x': 0.2 , 'y' : 0.9}

rangeselector = {'steps':["1m","2m","168hour"],}
#rangeslider = {"bgcolor":("blue",.3),"autorange":True}
#rangeslider = {"autorange":True}
rangeslider = True

In [ ]:
total_s = wind_df.sum(axis=1)
total_s.iplot(rangeselector=rangeselector, rangeslider =rangeslider)

In [ ]:
24*7

In [ ]:
total_s = wind_df.sum(axis=1)
total_s.iplot(rangeselector=range_selector)

In [ ]:
help(cl.tools.get_range_selector)

## colors

In [ ]:
px.colors.sequential.swatches_continuous()

In [ ]:
12 * 1 * 10 * 3500